# Worksheet 08

Name: **Bowen Li**  
UID: **U79057147** 

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [1]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[3.678730235051275, 5.07866867778319, 5.290684587728895, 5.8384938384222265, 5.735578786056836, 5.609941385281047, 3.3171927386854976, 4.848246839036331, 5.9333814655519355, 4.127952473017369]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [2]:
mean2 = 8
stdev2 = 1

s2 = np.random.normal(mean2, stdev2, 10).tolist()
print(s2)

[8.948714736267085, 7.501699543708617, 7.947504828669517, 8.142435836054752, 8.74435112534175, 7.914626525128113, 7.095639036373787, 7.353095450589233, 8.670445505049694, 7.6835938510425255]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [3]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1.pop()
        data.append(p1)
    else:
        p2 = s2.pop()
        data.append(p2)
print(data)

[4.127952473017369, 7.6835938510425255, 8.670445505049694, 7.353095450589233, 5.9333814655519355, 7.095639036373787, 4.848246839036331, 3.3171927386854976, 5.609941385281047, 7.914626525128113]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

$\mu_1=5$<br>
$\sigma_1=1$<br><br>
$\mu_2=8$<br>
$\sigma_2=1$

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [4]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / (len(s1) + len(s2)) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

C:\Users\libow\Programs\Anaconda\anaconda3\envs\datasci\lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\libow\Programs\Anaconda\anaconda3\envs\datasci\lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


[7.6835938510425255, 8.670445505049694, 7.353095450589233, 7.095639036373787, 7.914626525128113]
[4.127952473017369, 5.9333814655519355, 4.848246839036331, 3.3171927386854976, 5.609941385281047]
P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.74348007363667,  mean_2 = 4.767342980314437
var_1 = 0.29284810840463577,  var_2 = 0.9175838399162803


They are relatively close to the true values. $P(S_1)$ and $P(S_2)$ were on the mark while the means are relatively close to $8$ and $5$. One of the variances is very close to $1$, the true value, while the other is a bit further off.

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [5]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = [] # pdf_i[0] = p(p | s_0)

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( prob_s[0] * pdf_i[0] / prob_x )
    prob_s1_x.append( prob_s[1] * pdf_i[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  4.127952473017369
probability of observing that point if it came from cluster 0 =  1.4950470749433537e-10
probability of observing that point if it came from cluster 1 =  0.33330315573088826
point =  7.6835938510425255
probability of observing that point if it came from cluster 0 =  0.7327056898088594
probability of observing that point if it came from cluster 1 =  0.004045602701631971
point =  8.670445505049694
probability of observing that point if it came from cluster 0 =  0.16999762002554342
probability of observing that point if it came from cluster 1 =  0.00010337000348120718
point =  7.353095450589233
probability of observing that point if it came from cluster 0 =  0.5683082338776531
probability of observing that point if it came from cluster 1 =  0.010897086646964172
point =  5.9333814655519355
probability of observing that point if it came from cluster 0 =  0.002742154286666259
probability of observing that point if it came from cluster 1 =  0.19852970715353432
point 

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [6]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x)/ len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]
var = [ sum([x[0] * (x[1] - mean[0])**2 for x in zip(prob_s0_x, data)]) / sum(prob_s0_x) , sum([x[0] * (x[1] - mean[1])**2 for x in zip(prob_s1_x, data)]) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.5,  P(S_2) = 0.5
mean_1 = 7.746824811809375,  mean_2 = 4.80503285691634
var_1 = 0.3014996689200326,  var_2 = 1.001634174184171


After one iteration, these values are closer to the true values but not by much from the original K means result.

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [7]:
prob_s0_x_iter2 = [] # P(S_0 | X_i)
prob_s1_x_iter2 = [] # P(S_1 | X_i)
prob_x_iter2 = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = [] # pdf_i[0] = p(p | s_0)

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]**.5))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_c[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x_iter2 = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x_iter2.append( prob_c[0] * pdf_i[0] / prob_x_iter2 )
    prob_s1_x_iter2.append( prob_c[1] * pdf_i[1] / prob_x_iter2 )

probs_iter2 = zip(data, prob_s0_x_iter2, prob_s1_x_iter2)
for p in probs_iter2:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  4.127952473017369
probability of observing that point if it came from cluster 0 =  2.6856221126458087e-10
probability of observing that point if it came from cluster 1 =  0.3170804017410493
point =  7.6835938510425255
probability of observing that point if it came from cluster 0 =  0.7217504777571022
probability of observing that point if it came from cluster 1 =  0.006370659888257259
point =  8.670445505049694
probability of observing that point if it came from cluster 0 =  0.1765475048883936
probability of observing that point if it came from cluster 1 =  0.00022980627089209474
point =  7.353095450589233
probability of observing that point if it came from cluster 0 =  0.5618437148253459
probability of observing that point if it came from cluster 1 =  0.015595543777686903
point =  5.9333814655519355
probability of observing that point if it came from cluster 0 =  0.0031099638990570275
probability of observing that point if it came from cluster 1 =  0.2111256868251325
point = 

The differences with the K means result are extremely small.

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [8]:
assignments = zip(data, [int(prob_s0_x_iter2[i] > prob_s1_x_iter2[i]) for i in range(len(data))])
for datum, cluster in assignments:
    print(datum, "assigned to cluster", cluster)

4.127952473017369 assigned to cluster 0
7.6835938510425255 assigned to cluster 1
8.670445505049694 assigned to cluster 1
7.353095450589233 assigned to cluster 1
5.9333814655519355 assigned to cluster 0
7.095639036373787 assigned to cluster 1
4.848246839036331 assigned to cluster 0
3.3171927386854976 assigned to cluster 0
5.609941385281047 assigned to cluster 0
7.914626525128113 assigned to cluster 1
